In [3]:
from copy import deepcopy
import os
import sys
import argparse
import yaml
import pandas as pd
import subprocess
import shutil
import itertools
import numpy as np

'''
Script to search for optimal hyperparameter configuration of a model using grid search.

It takes an algorithm template folder (containing yaml files) 
with the following folder structure: 

algorithm_template_yaml_folder
│
├── algorithm1
│   ├── configs
│   │   ├── hyper_parameters.yaml
│   │   ├── network.yaml
│   │   ├── transforms_infer.yaml
│   │   ├── transforms_validate.yaml
│   │   └── transforms_train.yaml
│   ├── docs
│   │   └── algorithm1.md
│   └── scripts
│   │   ├── __init__.py
│   │   ├── infer.py
│   │   ├── train.py
│   │   └── validate.py
│
├── algorithm2
│   ├── configs
...

It reads the hyper_parameters.yaml file and performs a grid search over the hyperparameters.
This means, it searches for the given arguments in the hyper_parameters.yaml file and makes a grid search in the given range for that argument.
It changes the hyperparameters in the hyper_parameters.yaml file and runs the training with the new hyperparameters.
It always saves the best hyperparameters in the hyper_parameters.yaml file.

The script will run the training script for each hyperparameter configuration and save the results in a csv file.

The script will also save the best hyperparameters in the hyper_parameters.yaml file.

Arguments:
    -t, --template_folder: Path to the algorithm template folder
    -hl, --hyperparameter_list: List of hyperparameters to search for
    -r, --range: Range of the hyperparameters to search for

Example:
    python3 grid_search.py -t algorithm_template_yaml_folder -hl hyperparameter1 hyperparameter2 -r 1 2 3 4 5



'''



def get_hyperparameters(hyperparameter_file):
    """
    Reads the hyperparameters from a YAML file.

    Args:
        hyperparameter_file (str): The path to the YAML file containing the hyperparameters.

    Returns:
        dict: A dictionary containing the hyperparameters.

    Examples:
        >>> get_hyperparameters('configs/hyper_parameters.yaml')
        {'learning_rate': 0.001, 'batch_size': 32, 'epochs': 10}
    """
    with open(hyperparameter_file, 'r') as file:
        hyperparameters = yaml.load(file, Loader=yaml.FullLoader)
        
    # return deepcopy of hyperparameters to avoid changing the original hyperparameters
    return deepcopy(hyperparameters)

def set_hyperparameters(hyperparameter_file, hyperparameters):
    """
    Set the hyperparameters in a YAML file.

    Args:
        hyperparameter_file (str): The path to the YAML file to write the hyperparameters to.
        hyperparameters (dict): A dictionary containing the hyperparameters to be written.

    Returns:
        None
    """
    with open(hyperparameter_file, 'w') as file:
        yaml.dump(hyperparameters, file)

def run_training_script(script_path):
    # run the training scripts passing the training script path and all the yaml files
    subprocess.run(['python3', script_path, 'configs/hyper_parameters.yaml', 'configs/network.yaml', 'configs/transforms_train.yaml', 'configs/transforms_validate.yaml', 'configs/transforms_infer.yaml'])

def get_results(results_file):
    results = pd.read_csv(results_file)
    return results

def save_results(results_file, results):
    results.to_csv(results_file, index=False)

def get_best_hyperparameters(results):
    """
    Returns the best hyperparameters based on the minimum loss value in the results DataFrame.

    Parameters:
        results (pandas.DataFrame): DataFrame containing the hyperparameters and corresponding loss values.

    Returns:
        pandas.Series: Series containing the best hyperparameters.

    """
    best_hyperparameters = results.loc[results['loss'].idxmin()]
    return best_hyperparameters

def grid_search(hyperparameters_file: str, hyperparameters, hyperparameter_list, hyperparameter_range, results_file, script_path):
    """
    Perform a grid search over a range of hyperparameters.

    Args:
        hyperparameters_file (str): Path to the file containing hyperparameters.
        hyperparameters (dict): Dictionary of hyperparameters.
        hyperparameter_list (list): List of hyperparameters to be tuned.
        hyperparameter_range (list): List of values for each hyperparameter in the grid search.
        results_file (str): Path to the file to store the results.
        script_path (str): Path to the training script.

    Returns:
        dict: The best hyperparameters found during the grid search.

    """
    results = get_results(results_file)
    for hyperparameter in hyperparameter_list:
        for value in hyperparameter_range:
            hyperparameters[hyperparameter] = value
            set_hyperparameters(hyperparameters_file, hyperparameters)
            run_training_script(script_path)
            new_results = get_results(results_file)
            # TODO: in training script, we need to generate some results
            results = pd.concat([results, new_results])
            save_results(results_file, results)
    best_hyperparameters = get_best_hyperparameters(results)
    return best_hyperparameters

def main(template_folder, hyperparameter_list, hyperparameter_range, algorithm_list):
    """
    Perform grid search for hyperparameters on each algorithm in the template folder.

    Args:
        template_folder (str): Path to the folder containing algorithm templates.
        hyperparameter_list (list): List of hyperparameters to be tuned.
        hyperparameter_range (dict): Dictionary specifying the range of values for each hyperparameter.
        algorithm_list (list): List of algorithms to perform grid search on.

    Returns:
        None
    """

   
    all_algorithms = algorithm_list if algorithm_list is not None else os.listdir(template_folder)

    for algorithm in all_algorithms:
        algorithm_folder = os.path.join(template_folder, algorithm)
        hyperparameters_file = os.path.join(algorithm_folder, 'configs', 'hyper_parameters.yaml')
        hyperparameters = get_hyperparameters(hyperparameters_file)
        # TODO: adapt results file (might depend on working_dir) 
        results_file = os.path.join(algorithm_folder, 'results.csv')
        script_path = os.path.join(algorithm_folder, 'scripts', 'train.py')
        best_hyperparameters = grid_search(hyperparameters_file, hyperparameters, hyperparameter_list, hyperparameter_range, results_file, script_path)
        # save the best hyperparameters in the hyperparameters file
        set_hyperparameters(hyperparameters_file, best_hyperparameters)
    return


In [ ]:

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Search for optimal hyperparameter configuration of a model using grid search.')
    parser.add_argument('-t', '--template_folder', type=str, help='Path to the algorithm template folder', required=True)
    parser.add_argument('-hl', '--hyperparameter_list', nargs='+', help='List of hyperparameters to search for', required=True)
    parser.add_argument('-r', '--range', nargs='+', help='Range of the hyperparameters to search for', required=True)
    parser.add_argument('-a', '--algorithm_list', default = None, nargs='+', help='List of algorithms to search for', required=False)
    args = parser.parse_args()
    main(args.template_folder, args.hyperparameter_list, args.range, args.algorithm_list)
    sys.exit(0)

    

In [4]:
hyps = get_hyperparameters('/var/data/student_home/lia/phuse_thesis_2024/monai_segmentation/DNN_models/algorithm_templates_yaml/UNET/configs/hyper_parameters.yaml')
print(hyps)

{'bundle_root': None, 'ckpt_path': "$@bundle_root + '/model'", 'mlflow_tracking_uri': "$@ckpt_path + '/mlruns/'", 'mlflow_experiment_name': 'Auto3DSeg', 'data_file_base_dir': None, 'data_list_file_path': None, 'modality': 'mri', 'fold': 0, 'class_names': None, 'class_index': None, 'debug': False, 'ckpt_save': True, 'cache_rate': None, 'roi_size': [180, 240, 240], 'auto_scale_allowed': True, 'auto_scale_batch': True, 'auto_scale_roi': False, 'auto_scale_filters': False, 'quick': False, 'channels_last': True, 'validate_final_original_res': True, 'calc_val_loss': False, 'log_output_file': None, 'cache_class_indices': None, 'early_stopping_fraction': 0.001, 'stop_on_lowacc': False, 'training': {'amp': True, 'determ': False, 'input_channels': None, 'learning_rate': 0.0001, 'num_images_per_batch': 2, 'num_iterations': 40000, 'num_iterations_per_validation': 500, 'num_patches_per_image': 1, 'num_sw_batch_size': 2, 'output_classes': None, 'overlap_ratio': 0.625, 'patch_size': None, 'patch_size